In [14]:
import warnings
warnings.filterwarnings("ignore", message="...")

In [3]:
cd

/Users/sm


In [4]:
cd myenv4/OneFormer/

/Users/sm/myenv4/OneFormer


In [15]:
#detectron2 모델 저장
#build_detectron2.ipynb 실행
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="oneformer")

import numpy as np
import cv2
import torch
import imutils

from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.data import MetadataCatalog

from oneformer import (
    add_oneformer_config,
    add_common_config,
    add_swin_config,
    add_dinat_config,
    add_convnext_config,
)

In [17]:
SWIN_CFG_DICT = {"cityscapes": "configs/cityscapes/oneformer_swin_large_IN21k_384_bs16_90k.yaml",
            "coco": "configs/coco/oneformer_swin_large_IN21k_384_bs16_100ep.yaml",
            "ade20k": "configs/ade20k/oneformer_swin_large_IN21k_384_bs16_160k.yaml",}

DINAT_CFG_DICT = {"cityscapes": "configs/cityscapes/oneformer_dinat_large_bs16_90k.yaml",
            "coco": "configs/coco/oneformer_dinat_large_bs16_100ep.yaml",
            "ade20k": "configs/ade20k/oneformer_dinat_large_IN21k_384_bs16_160k.yaml",}

def setup_cfg(dataset, model_path, use_swin):
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_common_config(cfg)
    add_swin_config(cfg)
    add_dinat_config(cfg)
    add_convnext_config(cfg)
    add_oneformer_config(cfg)
    if use_swin:
      cfg_path = SWIN_CFG_DICT[dataset]
    else:
      cfg_path = DINAT_CFG_DICT[dataset]
    cfg.merge_from_file(cfg_path)
    cfg.MODEL.DEVICE = 'cpu'
    cfg.MODEL.WEIGHTS = model_path
    cfg.freeze()
    return cfg

def setup_modules(dataset, model_path, use_swin):
    cfg = setup_cfg(dataset, model_path, use_swin)
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get(
        cfg.DATASETS.TEST_PANOPTIC[0] if len(cfg.DATASETS.TEST_PANOPTIC) else "__unused"
    )
    if 'cityscapes_fine_sem_seg_val' in cfg.DATASETS.TEST_PANOPTIC[0]:
        from cityscapesscripts.helpers.labels import labels
        stuff_colors = [k.color for k in labels if k.trainId != 255]
        metadata = metadata.set(stuff_colors=stuff_colors)
    
    return predictor, metadata

def panoptic_run(img, predictor, metadata):
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.IMAGE)
    predictions = predictor(img, "panoptic")
    panoptic_seg, segments_info = predictions["panoptic_seg"]
    out = visualizer.draw_panoptic_seg_predictions(
    panoptic_seg.to(cpu_device), segments_info, alpha=0.5
)
    return out

def instance_run(img, predictor, metadata):
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.IMAGE)
    predictions = predictor(img, "instance")
    instances = predictions["instances"].to(cpu_device)
    out = visualizer.draw_instance_predictions(predictions=instances, alpha=0.5)
    return out

def semantic_run(img, predictor, metadata):
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.IMAGE)
    predictions = predictor(img, "semantic")
    out = visualizer.draw_sem_seg(
        predictions["sem_seg"].argmax(dim=0).to(cpu_device), alpha=0.5
    )
    return out

TASK_INFER = {"panoptic": panoptic_run, 
              "instance": instance_run, 
              "semantic": semantic_run}

In [18]:
from demo.defaults import DefaultPredictor
from demo.visualizer import Visualizer, ColorMode

In [8]:
use_swin = False
import os
import subprocess
if not use_swin:
  if not os.path.exists("250_16_dinat_l_oneformer_cityscapes_90k.pth"):
    subprocess.run('wget https://shi-labs.com/projects/oneformer/cityscapes/250_16_dinat_l_oneformer_cityscapes_90k.pth', shell=True)
  predictor, metadata = setup_modules("cityscapes", "250_16_dinat_l_oneformer_cityscapes_90k.pth", use_swin)
else:
  if not os.path.exists("250_16_swin_l_oneformer_cityscapes_90k.pth"):
    subprocess.run('wget https://shi-labs.com/projects/oneformer/cityscapes/250_16_swin_l_oneformer_cityscapes_90k.pth', shell=True)
  predictor, metadata = setup_modules("cityscapes", "250_16_swin_l_oneformer_cityscapes_90k.pth", use_swin)

Loading config configs/cityscapes/Base-Cityscapes-UnifiedSegmentation.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[03/23 15:40:29 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from 250_16_dinat_l_oneformer_cityscapes_90k.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  text_encoder.positional_embedding
  text_encoder.transformer.resblocks.0.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.0.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.0.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.0.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.1.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.1.attn.out_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_1.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_fc.{bias, weight}
  text_encoder.transformer.resblocks.1.mlp.c_proj.{bias, weight}
  text_encoder.transformer.resblocks.1.ln_2.{bias, weight}
  text_encoder.transformer.resblocks.2.attn.{in_proj_bias, in_proj_weight}
  text_encoder.transformer.resblocks.2.

## Save Output Image
#### Input = "samples/aachen/"
#### Output = "./output_img/"

In [9]:
#Panoptic segmentation된 이미지 파일 저장
from matplotlib import pyplot as plt
import os

input_path = "samples/aachen/"
output_path = "./output_img/"
os.makedirs(output_path, exist_ok=True)

for filename in os.listdir(input_path):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        input_filepath = os.path.join(input_path, filename)
        img = cv2.imread(input_filepath)
        img = imutils.resize(img, width=512)

        output_filename = f"{filename.split('.')[0]}_output.jpg"
        output_filepath = os.path.join(output_path, output_filename)

        task = "panoptic"
        cpu_device = torch.device('cpu')
        out = TASK_INFER[task](img, predictor, metadata).get_image()
        cv2.imwrite(output_filepath, out[:, :, ::-1])
        print(f"Output image saved at {output_filepath}")

Output image saved at ./output_img/aachen_1_output.jpg
Output image saved at ./output_img/aachen_0_output.jpg
Output image saved at ./output_img/aachen_2_output.jpg
Output image saved at ./output_img/aachen_3_output.jpg


## Save Output Json File
#### Input = "samples/aachen/"
#### Output = "./output_json/"

In [10]:
# prediction 결과를 json으로 변환하는 함수 정의

import json
    
def convert_output_to_json(output, file_path):
    json_output = {}

    # Add 'sem_seg' field to json_output
    json_output['sem_seg'] = output['sem_seg'].tolist()

    # Add 'panoptic_seg' field to json_output
    panoptic_seg = output['panoptic_seg']
    panoptic_seg_dict = {}
    panoptic_seg_dict['image_mask'] = panoptic_seg[0].tolist()
    categories = []
    for category in panoptic_seg[1]:
        category_dict = {}
        category_dict['id'] = category['id']
        category_dict['isthing'] = category['isthing']
        category_dict['category_id'] = category['category_id']
        categories.append(category_dict)
    panoptic_seg_dict['categories'] = categories
    json_output['panoptic_seg'] = panoptic_seg_dict

    # Add 'instances' field to json_output
    instances_dict = {}
    instances_dict['num_instances'] = len(output['instances'])
    instances_dict['image_height'] = output['instances'].image_size[0]
    instances_dict['image_width'] = output['instances'].image_size[1]
    
    fields = predictions['instances'].get_fields()
    instances_dict['pred_masks'] = fields['pred_masks'].tolist()
    instances_dict['pred_boxes'] = fields['pred_boxes'].tensor.tolist()
    instances_dict['scores'] = fields['scores'].tolist()
    instances_dict['pred_classes'] = fields['pred_classes'].tolist()

    json_output['instances'] = instances_dict
    
    with open(file_path, "w") as f:
        json.dump(json_output,f)

In [12]:
#Panoptic segmentation된 결과물을 json 파일로 저장
from matplotlib import pyplot as plt
import os

input_path = "samples/aachen"
output_path = "./output_json/"
os.makedirs(output_path, exist_ok=True)

img_files = os.listdir(input_path)
img_files.sort()

for filename in os.listdir(input_path):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        input_filepath = os.path.join(input_path, filename)
        img = cv2.imread(input_filepath)
        img = imutils.resize(img, width=512)

        output_filename = f"{filename.split('.')[0]}_output.json"
        output_filepath = os.path.join(output_path, output_filename)
        
        task = "panoptic"
        cpu_device = torch.device('cpu')
        # out = TASK_INFER[task](img, predictor, metadata).get_image()
        predictions = predictor(img, task)
        
        convert_output_to_json(predictions, output_filepath)
        print(f"Output json saved at {output_filepath}")

Output json saved at ./output_json/aachen_1_output.json
Output json saved at ./output_json/aachen_0_output.json
Output json saved at ./output_json/aachen_2_output.json
Output json saved at ./output_json/aachen_3_output.json
